In [42]:
from options import blackscholes as bs
from bizdays.bizdays import Calendar #pip3 install https://github.com/wilsonfreitas/python-bizdays

cal = Calendar.load(filename=r"C:\Users\maike\anaconda3\Lib\bizdays\ANBIMA.cal")

In [84]:
asset_price = 33.85
strike_price = 31.01
time_to_expiration = cal.bizdays('2022-04-12', '2022-05-20')
risk_free_rate = 11.75
qtd = 10000

In [85]:
bsm = {'short_petr4': bs.black_scholes(K=33.76,
                                        St=asset_price,
                                        r=risk_free_rate,
                                        t=time_to_expiration,
                                        v=31.30,
                                        type='c')}
bsm['short_petr4']['greeks']

{'delta': 0.5782157973095728,
 'gamma': 0.11496483723977133,
 'theta': -0.033981308417225554,
 'vega': 0.0425401992365655,
 'rho': 0.018532757514520395}

In [86]:
bsm['a'] = bs.black_scholes(K=34.51,
                        St=asset_price,
                        r=risk_free_rate,
                        t=time_to_expiration,
                        v=32.95,
                        type='c')

bsm['b'] = bs.black_scholes(K=35.26,
                        St=asset_price,
                        r=risk_free_rate,
                        t=time_to_expiration,
                        v=33.66,
                        type='c')

In [87]:
bsm

{'short_petr4': {'value': {'option value': 1.610085917163115,
   'intrinsic value': 0.09000000000000341,
   'time value': 1.5200859171631116},
  'greeks': {'delta': 0.5782157973095728,
   'gamma': 0.11496483723977133,
   'theta': -0.033981308417225554,
   'vega': 0.0425401992365655,
   'rho': 0.018532757514520395}},
 'a': {'value': {'option value': 1.31496432880253,
   'intrinsic value': 0,
   'time value': 1.31496432880253},
  'greeks': {'delta': 0.4940209712555761,
   'gamma': 0.11134245473105785,
   'theta': -0.03466675982936925,
   'vega': 0.04337169189784432,
   'rho': 0.01589677715290344}},
 'b': {'value': {'option value': 1.0397762028222335,
   'intrinsic value': 0,
   'time value': 1.0397762028222335},
  'greeks': {'delta': 0.4163378582310132,
   'gamma': 0.10660029457749903,
   'theta': -0.03354462845811788,
   'vega': 0.04241921913964324,
   'rho': 0.013467649514116532}}}

In [88]:
import numpy as np

greeks = np.array([[bsm['a']['greeks']['gamma'], bsm['b']['greeks']['gamma']], [bsm['a']['greeks']['vega'], bsm['b']['greeks']['vega']]])
portfolio_greeks = [[bsm['short_petr4']['greeks']['gamma']*qtd], [bsm['short_petr4']['greeks']['vega']*qtd]]

# We need to round otherwise we can end up with a non-invertible matrix
inv = np.linalg.inv(greeks)
print(inv)

[[  425.78952313 -1070.01707043]
 [ -435.35011688  1117.61724204]]


In [89]:
w = np.dot(inv, portfolio_greeks)
print(w)

[[ 34320.83862493]
 [-25062.95183441]]


In [90]:
print(np.round(np.dot(greeks, w) - portfolio_greeks))

[[0.]
 [0.]]


In [91]:
# Greeks including delta
portfolio_greeks = [[bsm['short_petr4']['greeks']['delta']*-qtd],
                    [bsm['short_petr4']['greeks']['gamma']*-qtd],
                    [bsm['short_petr4']['greeks']['vega']*-qtd]]

greeks = np.array([[bsm['a']['greeks']['delta'], bsm['b']['greeks']['delta']],
                    [bsm['a']['greeks']['gamma'], bsm['b']['greeks']['gamma']],
                    [bsm['a']['greeks']['vega'], bsm['b']['greeks']['vega']]])

print(np.round(np.dot(greeks, w) + portfolio_greeks))

[[738.]
 [  0.]
 [  0.]]
